In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, callbacks
import pickle
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import pickle

# Set the random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Define the paths to the data directories
train_image_dir = 'fall_dataset/images/train/'
train_label_dir = 'fall_dataset/labels/train/'
val_image_dir = 'fall_dataset/images/val/'
val_label_dir = 'fall_dataset/labels/val/'

# Define the label dictionary
label_dict = {'Fall': 1, 'Not_Fall': 0}

# Define a function to read the data
def read_dataset(image_dir, label_dir):
    images = []
    labels = []
    for image_filename in os.listdir(image_dir):
        if image_filename.endswith('.jpg'):
            # Read the image
            img_array = cv2.imread(os.path.join(image_dir, image_filename))
            img_array = cv2.resize(img_array, (224, 224))
            images.append(img_array)
            
            # Read the label file and extract the class information
            label_filename = image_filename.replace('.jpg', '.txt')
            if os.path.exists(os.path.join(label_dir, label_filename)):
                with open(os.path.join(label_dir, label_filename), 'r') as f:
                    line = f.readline()
                class_id = int(line.split()[0])
                label = label_dict['Fall'] if class_id == 0 else label_dict['Not_Fall']
                labels.append(label)
                
    return np.array(images), np.array(labels)

# Read the training and validation data
train_images, train_labels = read_dataset(train_image_dir, train_label_dir)
val_images, val_labels = read_dataset(val_image_dir, val_label_dir)

# Convert the labels to numpy arrays
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

# Define the model
def create_model(learning_rate=1e-3, dropout_rate=0.25, filters1=32, filters2=64, filters3=128, dense1=128):
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(filters1, (3,3), activation='relu', input_shape=(224,224,3)))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Conv2D(filters2, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Conv2D(filters3, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(dense1, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Convert the Keras model to a scikit-learn estimator
estimator = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=1)

# Define the hyperparameters to search
params = {'learning_rate': [1e-3, 1e-4, 1e-5],
'dropout_rate': [0.25, 0.5],
'filters1': [32, 64, 128],
'filters2': [64, 128, 256],
'filters3': [128, 256, 512],
'dense1': [64, 128, 256]}

# Define the random search object
random_search = RandomizedSearchCV(estimator=estimator,
                                   param_distributions=params, 
                                   n_iter=10, cv=3, 
                                   verbose=1, n_jobs=1)

# Perform the random search to find the best hyperparameters
random_search.fit(train_images, train_labels)

# Print the best hyperparameters
print('Best hyperparameters:', random_search.best_params_)

<ipython-input-2-e4351ab2d050>:77: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=1)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/10
8/8 [==============================] - 122s 13s/step - loss: 8468.3486 - accuracy: 0.5863
Epoch 2/10
8/8 [==============================] - 100s 12s/step - loss: 3.2135 - accuracy: 0.5783
Epoch 3/10
8/8 [==============================] - 109s 14s/step - loss: 0.6625 - accuracy: 0.6908
Epoch 4/10
8/8 [==============================] - 75s 9s/step - loss: 0.6689 - accuracy: 0.6787
Epoch 5/10
8/8 [==============================] - 57s 7s/step - loss: 0.6621 - accuracy: 0.7309
Epoch 6/10
8/8 [==============================] - 56s 7s/step - loss: 0.6484 - accuracy: 0.7671
Epoch 7/10
8/8 [==============================] - 56s 7s/step - loss: 0.6157 - accuracy: 0.6787
Epoch 8/10
8/8 [==============================] - 56s 7s/step - loss: 0.5855 - accuracy: 0.6908
Epoch 9/10
8/8 [==============================] - 55s 7s/step - loss: 0.5704 - accuracy: 0.7269
Epoch 10/10
4/4 [==============================] - 7s 1s/step - lo

<h1 style="font-size:70px">Lightweight Best Run 1</h1>

In [74]:
# Import the necessary libraries
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import Adam

# Define the paths to the data directories
train_image_dir = 'fall_dataset/images/train'
train_label_dir = 'fall_dataset/labels/train'
val_image_dir = 'fall_dataset/images/val'
val_label_dir = 'fall_dataset/labels/val'

# Define the label dictionary
label_dict = {'Fall': 1, 'Not_Fall': 0}

# Define a function to read the data
def read_dataset(image_dir, label_dir):
    images = []
    labels = []
    for image_filename in os.listdir(image_dir):
        if image_filename.endswith('.jpg'):
            # Read the image
            img_array = cv2.imread(os.path.join(image_dir, image_filename))
            img_array = cv2.resize(img_array, (64, 64))
            images.append(img_array)
            
            # Read the label file and extract the class information
            label_filename = image_filename.replace('.jpg', '.txt')
            if os.path.exists(os.path.join(label_dir, label_filename)):
                with open(os.path.join(label_dir, label_filename), 'r') as f:
                    line = f.readline()
                class_id = int(line.split()[0])
                label = label_dict['Fall'] if class_id == 0 else label_dict['Not_Fall']
                labels.append(label)
                
    return np.array(images), np.array(labels)

# Define a CNN model with the best hyperparameters
def create_model(filters1, filters2, filters3, dense1, dropout_rate, learning_rate):
    model = Sequential([
        Conv2D(filters1, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(filters2, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(filters3, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(dense1, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define the best hyperparameters
best_hyperparameters = {'learning_rate': 0.0001, 
                        'filters1': 64, 
                        'filters2': 128, 
                        'filters3': 512, 
                        'dense1': 256, 
                        'dropout_rate': 0.5}

# Read the training and validation data
X_train, y_train = read_dataset(train_image_dir, train_label_dir)
X_val, y_val = read_dataset(val_image_dir, val_label_dir)

# Normalize the pixel values
X_train = X_train / 255.0
X_val = X_val / 255.0

# Create the model with the best hyperparameters
model = create_model(**best_hyperparameters)

# Train the model
model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))

# Make predictions on the validation set
y_pred = model.predict(X_val)
y_pred = (y_pred > 0.5).astype(int)

# Generate a classification report
print(classification_report(y_val, y_pred))

# Save the trained model to a pickle file
model.save('lightweight_augmented.h5')

Epoch 1/50
12/12 [==============================] - 28s 1s/step - loss: 0.6957 - accuracy: 0.5267 - val_loss: 0.6716 - val_accuracy: 0.7753
Epoch 2/50
12/12 [==============================] - 13s 1s/step - loss: 0.6832 - accuracy: 0.5829 - val_loss: 0.6581 - val_accuracy: 0.7753
Epoch 3/50
12/12 [==============================] - 14s 1s/step - loss: 0.6675 - accuracy: 0.6176 - val_loss: 0.6595 - val_accuracy: 0.6966
Epoch 4/50
12/12 [==============================] - 13s 1s/step - loss: 0.6465 - accuracy: 0.6631 - val_loss: 0.6350 - val_accuracy: 0.7079
Epoch 5/50
12/12 [==============================] - 13s 1s/step - loss: 0.6079 - accuracy: 0.7326 - val_loss: 0.6734 - val_accuracy: 0.5506
Epoch 6/50
12/12 [==============================] - 14s 1s/step - loss: 0.5622 - accuracy: 0.7380 - val_loss: 0.5078 - val_accuracy: 0.7640
Epoch 7/50
12/12 [==============================] - 15s 1s/step - loss: 0.5456 - accuracy: 0.7273 - val_loss: 0.6107 - val_accuracy: 0.6404
Epoch 8/50
12/12 [==

<h1 style="font-size:70px">Image Test</h1>

In [19]:
import cv2
import numpy as np
from keras.models import load_model
import os

model = load_model('/Final PCVK/lightweight_augmented.h5')
test_folder = '/Final PCVK/Photo Test/'

for filename in os.listdir(test_folder):
    img_path = os.path.join(test_folder, filename)
    img = cv2.imread(img_path)
    
    if img is None:
        print(f"\nDone")
        continue
    
    img = cv2.resize(img, (64, 64))
    
    if img is None:
        print(f"Failed to resize image: {img_path}")
        continue

    prediction = model.predict(np.expand_dims(img, axis=0))

    if prediction > 0.5:
        print(filename, ': Jatuh')
    else:
        print(filename, ': Tidak jatuh')

1/1 [==============================] - 0s 497ms/step
duduk 1.jpg : Tidak jatuh
1/1 [==============================] - 0s 101ms/step
duduk 2.jpg : Tidak jatuh
1/1 [==============================] - 0s 110ms/step
jatuh 1.jpg : Tidak jatuh
1/1 [==============================] - 0s 108ms/step
jatuh 2.jpg : Jatuh
1/1 [==============================] - 0s 91ms/step
jatuh 3.jpg : Jatuh
1/1 [==============================] - 0s 71ms/step
jatuh 4.jpg : Jatuh
1/1 [==============================] - 0s 72ms/step
lari 1.jpg : Tidak jatuh
1/1 [==============================] - 0s 85ms/step
lari 2.jpg : Tidak jatuh

Done


<h1 style="font-size:70px">Video Test</h1>

In [18]:
# Import the necessary libraries
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import os

# Load the trained model
model = load_model('lightweight_augmented.h5')

# Define the label dictionary
label_dict = {1: 'Fall', 0: 'Not_Fall'}

# Open the video file
video_path = '/Final PCVK/Video Test/Walking2.mp4'
cap = cv2.VideoCapture(video_path)

# Loop through each frame in the video
while True:
    # Read a frame from the video
    ret, frame = cap.read()
    
    # Break the loop if the end of the video is reached
    if not ret:
        break
    
    # Preprocess the frame
    input_image = cv2.resize(frame, (64, 64))
    input_image = np.expand_dims(input_image, axis=0) / 255
    
    # Make a prediction
    output = model.predict(input_image)[0][0]
    predicted_class = int(output > 0.5)
    predicted_label = label_dict[predicted_class]
    
    # If a fall is detected, save the current frame
    if predicted_label == 'Fall':
        cv2.imwrite('/Final PCVK/Video Test/Output/Fall Detected.jpg', frame)
        print('A fall was detected.');

# Release the video file
cap.release()

# Print whether a fall was detected or not
if 'Fall Detected.jpg' in os.listdir('/Final PCVK/Video Test/Output/'):
    print('\nA fall was detected.');
else:
    print('\nNo fall was detected.')

1/1 [==============================] - 0s 41ms/step

No fall was detected.


<h1 style="font-size:80">YOLO 1</h1>

In [12]:
import cv2
import numpy as np

# load the YOLO model
net = cv2.dnn.readNetFromDarknet("yolov3.cfg", "yolov3.weights")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# load the image
image = cv2.imread("/Final PCVK/Photo Test/duduk 2.jpg")

# define the minimum confidence threshold and non-maxima suppression threshold
confidence_threshold = 0.5
nms_threshold = 0.4

# get the output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# get the height, width, and number of channels of the input image
height, width, channels = image.shape

# create a blob from the input image
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)

# set the input for the network
net.setInput(blob)

# forward pass through the network
outputs = net.forward(output_layers)

# initialize empty lists for bounding boxes, confidences, and class IDs
boxes = []
confidences = []
class_ids = []

# loop over each output from the network
for output in outputs:
    # loop over each detection in the output
    for detection in output:
        # get the class ID and confidence of the detection
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        
        # filter out weak detections
        if confidence > confidence_threshold:
            # scale the bounding box coordinates to the input image size
            box = detection[0:4] * np.array([width, height, width, height])
            center_x, center_y, w, h = box.astype("int")
            x = int(center_x - (w / 2))
            y = int(center_y - (h / 2))
            
            # append the bounding box, confidence, and class ID to the lists
            boxes.append([x, y, int(w), int(h)])
            confidences.append(float(confidence))
            class_ids.append(class_id)
            
# apply non-maxima suppression to remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)

# define colors for bounding boxes based on class labels
CLASSES = ["person"]
COLORS = [[0, 255, 0]]

# loop over the remaining indices
for i in indices.flatten():
    # extract the bounding box coordinates
    x, y = boxes[i][0], boxes[i][1]
    w, h = boxes[i][2], boxes[i][3]
    
    # draw the bounding box and label on the image
    color = [int(c) for c in COLORS[0]]
    cv2.rectangle(image, (x, y), (x + w, y + h), color, 25)
    text = "{}: {:.4f}".format(CLASSES[0], confidences[i])
    cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# show the output image
# define the path and filename for the output image
output_path = "/Final PCVK/Photo Test/Output/output.jpg"

# save the output image to the specified path
cv2.imwrite(output_path, image)


True

<h1 style="font-size:80">YOLO 2</h1>

In [14]:
import os
import cv2
import numpy as np

# load the YOLO model
net = cv2.dnn.readNetFromDarknet("yolov3.cfg", "yolov3.weights")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# define the minimum confidence threshold and non-maxima suppression threshold
confidence_threshold = 0.5
nms_threshold = 0.4

# get the output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# define colors for bounding boxes based on class labels
CLASSES = ["person"]  # update this list with all the classes that the model is detecting
COLORS = [[255, 255, 0]]

# loop over each image in the folder
folder_path = "/Final PCVK/Photo Test/"
output_folder_path = "/Final PCVK/Photo Test/Output"
for file_name in os.listdir(folder_path):
    if file_name.endswith(".jpg") or file_name.endswith(".jpeg") or file_name.endswith(".png"):
        # load the image
        image_path = os.path.join(folder_path, file_name)
        image = cv2.imread(image_path)

        # get the height, width, and number of channels of the input image
        height, width, channels = image.shape

        # create a blob from the input image
        blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)

        # set the input for the network
        net.setInput(blob)

        # forward pass through the network
        outputs = net.forward(output_layers)

        # initialize empty lists for bounding boxes, confidences, and class IDs
        boxes = []
        confidences = []
        class_ids = []

        # loop over each output from the network
        for output in outputs:
            # loop over each detection in the output
            for detection in output:
                # get the class ID and confidence of the detection
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                # filter out weak detections
                if confidence > confidence_threshold:
                    class_name = CLASSES[class_id] if class_id < len(CLASSES) else "unknown"
                    if class_name == "person":
                        # scale the bounding box coordinates to the input image size
                        box = detection[0:4] * np.array([width, height, width, height])
                        center_x, center_y, w, h = box.astype("int")
                        x = int(center_x - (w / 2))
                        y = int(center_y - (h / 2))

                        # append the bounding box, confidence, and class ID to the lists
                        boxes.append([x, y, int(w), int(h)])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

        # apply non-maxima suppression to remove overlapping boxes
        indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)

        # loop over the remaining indices
        for i in np.array(indices).flatten():
            # extract the bounding box coordinates
            x, y = boxes[i][0], boxes[i][1]
            w, h = boxes[i][2], boxes[i][3]

            # draw the bounding box and label on the image
            color = [int(c) for c in COLORS[0]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 5)
            text = "{}: {:.4f}".format(CLASSES[class_ids[i]], confidences[i])
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


        # save the annotated image to the output folder
        output_path = os.path.join(output_folder_path, file_name)
        cv2.imwrite(output_path, image)